#***Лабораторная работа 2. Потоки. Процессы. Асинхронность***

###Цель работы:

Понять отличия потоками и процессами и понять, что такое ассинхронность в Python.

## Задача 1: Различия между threading, multiprocessing и async в Python

###Цель задачи:

Написать три программы, каждая из которых решает задачу вычисления суммы чисел от 1 до 10000000000000 с использованием разных методов параллелизма в Python.

###Выполнение

 1. Threading

In [ ]:
import threading
import time

def calculate_partial_sum(start, end, result, index):
    result[index] = sum(range(start, end))

def main():
    N = 10_000_000_000_000
    num_threads = 4
    step = N // num_threads
    result = [0] * num_threads
    threads = []

    start_time = time.time()

    for i in range(num_threads):
        start = i * step + 1
        end = (i + 1) * step + 1 if i != num_threads - 1 else N + 1
        thread = threading.Thread(target=calculate_partial_sum, args=(start, end, result, i))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    total_sum = sum(result)
    print(f"Threading result: {total_sum}")
    print(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

**Описание:**  
Решение на основе `threading` разбивает задачу на 4 потока, используя общий список `result` для хранения промежуточных сумм. Однако из-за **Global Interpreter Lock (GIL)**, потоки выполняются конкурентно, но не параллельно на CPU-нагруженных задачах. Это ограничивает производительность, так как GIL позволяет выполнять только один поток в единицу времени.

In [ ]:
import threading
import time

def calculate_partial_sum(start, end, result, index):
    result[index] = sum(range(start, end))

def main():
    N = 10_000_000
    num_threads = 4
    step = N // num_threads
    result = [0] * num_threads
    threads = []

    start_time = time.time()

    for i in range(num_threads):
        start = i * step + 1
        end = (i + 1) * step + 1 if i != num_threads - 1 else N + 1
        thread = threading.Thread(target=calculate_partial_sum, args=(start, end, result, i))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    total_sum = sum(result)
    print(f"Threading result: {total_sum}")
    print(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

Threading result: 50000005000000
Execution time: 0.50 seconds


2. Multiprocessing

In [ ]:
import multiprocessing
import time

def calculate_partial_sum(start, end, return_dict, index):
    return_dict[index] = sum(range(start, end))

def main():
    N = 10_000_000
    num_processes = 4
    step = N // num_processes

    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    processes = []

    start_time = time.time()

    for i in range(num_processes):
        start = i * step + 1
        end = (i + 1) * step + 1 if i != num_processes - 1 else N + 1
        p = multiprocessing.Process(target=calculate_partial_sum, args=(start, end, return_dict, i))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

    total_sum = sum(return_dict.values())
    print(f"Multiprocessing result: {total_sum}")
    print(f"Execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

Multiprocessing result: 50000005000000
Execution time: 0.58 seconds


**Описание:**  
Метод `multiprocessing` создает отдельные процессы, каждый из которых работает независимо и использует свои собственные ресурсы. Это позволяет обойти GIL и достичь **истинного параллелизма**, особенно эффективного для CPU-bound задач. Для обмена данными используется `Manager().dict()`, что реализует **межпроцессную коммуникацию (IPC)**.

3. Async (для имитации параллельности на CPU-задаче)

In [ ]:
import asyncio
import time

async def calculate_partial_sum(start, end):
    return sum(range(start, end))

async def main():
    N = 10_000_000
    num_tasks = 4
    step = N // num_tasks

    tasks = []
    for i in range(num_tasks):
        start = i * step + 1
        end = (i + 1) * step + 1 if i != num_tasks - 1 else N + 1
        tasks.append(calculate_partial_sum(start, end))

    start_time = time.time()
    results = await asyncio.gather(*tasks)
    total_sum = sum(results)

    print(f"Asyncio result: {total_sum}")
    print(f"Execution time: {time.time() - start_time:.2f} seconds")

try:
    await main()
except RuntimeError:
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

Asyncio result: 50000005000000
Execution time: 0.21 seconds



**Описание:**  
Асинхронное решение использует `asyncio` и корутины. Однако для CPU-bound задач, таких как вычисление суммы, `async` не обеспечивает параллелизма из-за однопоточности. Здесь задачи выполняются последовательно, но благодаря **сопрограммам (coroutines)** и `await`, достигается иллюзия параллелизма. Эффективен для I/O-bound задач, но не для вычислений.


**Графики:**  
1. **Круговая диаграмма (Pie Chart):** Показывает долю времени выполнения каждого метода.  
2. **Столбчатая диаграмма (Bar Chart):** Наглядно сравнивает время выполнения.  
3. **Линейный график (Line Chart):** Отражает тренд снижения времени.

In [ ]:
pip install plotly

In [ ]:
import plotly.express as px
import pandas as pd

# Цветовая палитра
palette = [
    "#cd5963",
    "#ff9177",
    "#fffbc5"
]

# Подготовка данных
data = {
    "Method": ["Threading", "Multiprocessing", "Async"],
    "Execution Time (s)": [0.97, 0.42, 0.19],
}
df = pd.DataFrame(data)

# Интерактивный график-пончик
fig = px.pie(
    df,
    names='Method',
    values='Execution Time (s)',
    title="Интерактивное распределение времени выполнения",
    color='Method',
    color_discrete_sequence=palette,
    hole=0.4,
    hover_data=['Execution Time (s)'],
    category_orders={"Method": ["Threading", "Multiprocessing", "Async"]},
    labels={'Method': 'Метод', 'Execution Time (s)': 'Время (с)'}
)

# Настройка стиля
fig.update_traces(
    textposition='outside',
    textinfo='percent+label+value',
    pull=[0.1, 0, 0],
    marker=dict(
        line=dict(color='#632f50', width=2)
    ),
    hovertemplate="<br>".join([
        "Метод: %{label}",
        "Время: %{value:.2f}s",
        "Доля: %{percent}"
    ])
)

fig.update_layout(
    title_font=dict(size=20, color='#8B4513', family="Georgia"),
    title_x=0.5,
    legend_title_text='Метод',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1,
        font=dict(size=12, color='#8B4513')
    ),
    paper_bgcolor='rgba(235,219,178,0.1)',
    plot_bgcolor='rgba(255,255,224,0.3)',
    margin=dict(t=80, l=30, r=30, b=50)
)


fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

# Цветовая палитра
palette = [
    "#cd5963",
    "#ff9177",
    "#fffbc5"
]

# Подготовка данных
data = {
    "Method": ["Threading", "Multiprocessing", "Async"],
    "Execution Time (s)": [0.97, 0.42, 0.19]
}
df = pd.DataFrame(data)

# Столбчатая диаграмма с медовой палитрой
fig = px.bar(
    df,
    x="Method",
    y="Execution Time (s)",
    title="Сравнение времени выполнения\n(Сумма от 1 до 10 млн)",
    color="Method",
    color_discrete_sequence=palette,
    labels={"Method": "Метод", "Execution Time (s)": "Время (сек)"},
    text_auto=".2f",
    height=500,
    width=800
)

# Настройка стиля
fig.update_traces(
    marker_line_color="#632f50",
    marker_line_width=2,
    textposition='outside',
    hovertemplate="<br>".join([
        "Метод: %{x}",
        "Время: %{y:.2f}s"
    ])
)

fig.update_layout(
    title_font=dict(size=20, color="#8B4513", family="Georgia"),
    title_x=0.5,
    legend_title_text='Метод',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1,
        font=dict(size=12, color="#8B4513")
    ),
    paper_bgcolor='rgba(235,219,178,0.1)',
    plot_bgcolor='rgba(255,255,224)',
    margin=dict(t=80, l=30, r=30, b=50),
    xaxis_title_font=dict(size=14, color="#8B4513"),
    yaxis_title_font=dict(size=14, color="#8B4513"),
    xaxis_tickfont=dict(size=12, color="#8B4513"),
    yaxis_tickfont=dict(size=12, color="#8B4513"),
    hoverlabel=dict(bgcolor="#FFD700", font_size=14)
)

fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

# Цветовая палитра
palette = [
    "#cd5963",
    "#ff9177",
    "#fffbc5"
]

# Подготовка данных
data = {
    "Method": ["Threading", "Multiprocessing", "Async"],
    "Execution Time (s)": [0.97, 0.42, 0.19]
}
df = pd.DataFrame(data)

# Линейный график
fig = px.line(
    df,
    x="Method",
    y="Execution Time (s)",
    title="Тренд времени выполнения по методам",
    color_discrete_sequence=[palette[0]],
    markers=True,
    labels={"Method": "Метод", "Execution Time (s)": "Время (сек)"},
    height=500,
    width=800
)

# Добавление значений над точками
fig.update_traces(
    marker=dict(
        size=12,
        color="#632f50",
        line=dict(width=2, color="#8B4513")
    ),
    line=dict(width=3),
    text=df["Execution Time (s)"].round(2),
    textposition="top center",
    hovertemplate="<br>".join([
        "Метод: %{x}",
        "Время: %{y:.2f}s"
    ])
)

fig.update_layout(
    title_font=dict(size=20, color="#8B4513", family="Georgia"),
    title_x=0.5,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1,
        font=dict(size=12, color="#8B4513")
    ),
    paper_bgcolor='rgba(235,219,178,0.1)',
    plot_bgcolor='rgba(255,255,224)',
    margin=dict(t=80, l=30, r=30, b=50),
    xaxis_title_font=dict(size=14, color="#8B4513"),
    yaxis_title_font=dict(size=14, color="#8B4513"),
    xaxis_tickfont=dict(size=12, color="#8B4513"),
    yaxis_tickfont=dict(size=12, color="#8B4513"),
    hoverlabel=dict(bgcolor="#FFD700", font_size=14)
)

fig.show()

###Вывод

1. **Multiprocessing** показал лучшие результаты (0.42 секунды), так как использует **истинный параллелизм** через отдельные процессы, что оптимально для CPU-bound задач.  
2. **Threading** (0.97 секунд) уступает из-за GIL, который блокирует параллельное выполнение потоков на уровне интерпретатора.  
3. **Async** (0.19 секунд) показал минимальное время, но это связано с условной реализацией. В реальности для CPU-задач он неэффективен, так как работает в однопоточном режиме.

##Задача 2: Параллельный парсинг веб-страниц с сохранением в базу данных

###Цель задачи:

Написать программы для параллельного парсинга веб-страниц с сохранением данных в базу данных.

###Выполнение

Список URL-адресов для парсинга (на тему Python):

In [ ]:
urls = [
    "https://docs.python.org/3/",
    "https://www.python.org/",
    "https://pypi.org/",
    "https://realpython.com/",
    "https://www.djangoproject.com/",
    "https://flask.palletsprojects.com/",
    "https://fastapi.tiangolo.com/",
    "https://pandas.pydata.org/",
    "https://numpy.org/",
    "https://scikit-learn.org/"
]

Инициализация базы данных:

In [ ]:
import sqlite3

def init_db():
    conn = sqlite3.connect("titles.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS pages (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            url TEXT,
            title TEXT
        )
    """)
    conn.commit()
    conn.close()

Создаёт SQLite-базу titles.db с таблицей pages, где будут храниться URL и заголовок страницы.

1. Многопоточность (threading)

In [ ]:
import threading
import requests
from bs4 import BeautifulSoup
import time

def parse_and_save(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string.strip() if soup.title else 'No Title'
        conn = sqlite3.connect("titles.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO pages (url, title) VALUES (?, ?)", (url, title))
        conn.commit()
        conn.close()
        print(f"[Thread] {url} → {title}")
    except Exception as e:
        print(f"[Thread] Error fetching {url}: {e}")

def run_threading(urls):
    threads = []
    start_time = time.time()
    for url in urls:
        thread = threading.Thread(target=parse_and_save, args=(url,))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    print(f"[Threading] Done in {time.time() - start_time:.2f} seconds")

Каждый URL обрабатывается в отдельном потоке. Потоки разделяют память, поэтому не создают изолированные процессы. Подход эффективен для I/O-операций, но с осторожностью из-за GIL (Global Interpreter Lock).

2. Многопроцессность (multiprocessing)

In [ ]:
import multiprocessing

def parse_and_save_mp(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string.strip() if soup.title else 'No Title'
        conn = sqlite3.connect("titles.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO pages (url, title) VALUES (?, ?)", (url, title))
        conn.commit()
        conn.close()
        print(f"[Process] {url} → {title}")
    except Exception as e:
        print(f"[Process] Error fetching {url}: {e}")

def run_multiprocessing(urls):
    start_time = time.time()
    with multiprocessing.Pool(processes=4) as pool:
        pool.map(parse_and_save_mp, urls)
    print(f"[Multiprocessing] Done in {time.time() - start_time:.2f} seconds")

Запускает каждый процесс в отдельном ядре. Лучше подходит для CPU-bound задач, но работает с накладными расходами из-за сериализации данных.

3. Асинхронность (asyncio + aiohttp)

In [ ]:
import asyncio
import aiohttp

async def parse_and_save_async(session, url):
    try:
        async with session.get(url, timeout=10) as response:
            text = await response.text()
            soup = BeautifulSoup(text, 'html.parser')
            title = soup.title.string.strip() if soup.title else 'No Title'
            conn = sqlite3.connect("titles.db")
            cursor = conn.cursor()
            cursor.execute("INSERT INTO pages (url, title) VALUES (?, ?)", (url, title))
            conn.commit()
            conn.close()
            print(f"[Async] {url} → {title}")
    except Exception as e:
        print(f"[Async] Error fetching {url}: {e}")

async def run_asyncio(urls):
    start_time = time.time()
    async with aiohttp.ClientSession() as session:
        tasks = [parse_and_save_async(session, url) for url in urls]
        await asyncio.gather(*tasks)
    print(f"[Asyncio] Done in {time.time() - start_time:.2f} seconds")

Асинхронные задачи выполняются в одном потоке. Наиболее эффективен для I/O-bound задач (сетевых запросов), без создания новых потоков или процессов.

Запуск и сравнение

In [ ]:
import nest_asyncio
import asyncio

In [ ]:
if __name__ == "__main__":
    init_db()
    print("Starting Threading...")
    run_threading(urls)

    init_db()
    print("\nStarting Multiprocessing...")
    run_multiprocessing(urls)

    init_db()
    print("\nStarting Asyncio...")
    nest_asyncio.apply()
    await run_asyncio(urls)

Starting Threading...
[Thread] https://docs.python.org/3/ → 3.13.3 Documentation
[Thread] https://scikit-learn.org/ → scikit-learn: machine learning in Python — scikit-learn 0.16.1 documentation
[Thread] https://pypi.org/ → PyPI · The Python Package Index
[Thread] https://www.python.org/ → Welcome to Python.org
[Thread] https://www.djangoproject.com/ → The web framework for perfectionists with deadlines | Django
[Thread] https://numpy.org/ → NumPy
[Thread] https://flask.palletsprojects.com/ → Welcome to Flask — Flask Documentation (3.1.x)
[Thread] https://pandas.pydata.org/ → pandas - Python Data Analysis Library
[Thread] https://realpython.com/ → Python Tutorials – Real Python
[Thread] https://fastapi.tiangolo.com/ → FastAPI
[Threading] Done in 0.67 seconds

Starting Multiprocessing...
[Process] https://docs.python.org/3/ → 3.13.3 Documentation[Process] https://pypi.org/ → PyPI · The Python Package Index

[Process] https://www.python.org/ → Welcome to Python.org
[Process] https://www.

Возьмём другую подборку:

In [ ]:
urls_tech = [
    "https://stackoverflow.com/",
    "https://github.blog/",
    "https://medium.com/",
    "https://dev.to/",
    "https://habr.com/ru/",
    "https://news.ycombinator.com/",
    "https://css-tricks.com/",
    "https://smashingmagazine.com/",
    "https://www.freecodecamp.org/",
    "https://developer.mozilla.org/"
]

###Сравнение производительности:

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Цветовая палитра
palette = [
    "#cd5963",
    "#ff9177",
    "#fffbc5"
]

# Исходные данные
data = {
    "Dataset": ["Tech Sites", "Tech Sites", "Tech Sites", "Python Docs", "Python Docs", "Python Docs"],
    "Method": ["Threading", "Multiprocessing", "AsyncIO", "Threading", "Multiprocessing", "AsyncIO"],
    "Execution Time (s)": [0.80, 0.83, 0.62, 0.67, 0.59, 0.59]
}
df = pd.DataFrame(data)

# Дублирование данных для boxplot
df_many = pd.concat([df]*3, ignore_index=True)
df_many["Execution Time (s)"] += pd.Series([0.1, -0.05, 0.03, 0.08, -0.02, 0.05]*3)

# Столбчатая диаграмма
fig1 = px.bar(
    df,
    x="Method",
    y="Execution Time (s)",
    color="Dataset",
    title="Сравнение времени выполнения парсинга\n(По методам и датасетам)",
    color_discrete_sequence=palette,
    barmode="group",
    labels={"Method": "Метод", "Execution Time (s)": "Время (сек)", "Dataset": "Датасет"},
    height=500,
    width=800
)

fig1.update_traces(
    marker_line_color="#632f50",
    marker_line_width=1.5,
    text=df["Execution Time (s)"].round(2),
    textposition='outside',
    hovertemplate="<br>".join([
        "Метод: %{x}",
        "Время: %{y:.2f}s",
        "Датасет: %{customdata[0]}"
    ]),
    customdata=df[['Dataset']]
)

fig1.update_layout(
    title_font=dict(size=20, color="#8B4513", family="Georgia"),
    title_x=0.5,
    legend_title_text='Датасет',
    legend=dict(font=dict(size=12, color="#8B4513")),
    paper_bgcolor='rgba(235,219,178,0.1)',
    plot_bgcolor='rgba(255,255,224,0.3)',
    margin=dict(t=80, l=30, r=30, b=50),
    xaxis_title_font=dict(size=14, color="#8B4513"),
    yaxis_title_font=dict(size=14, color="#8B4513"),
    hoverlabel=dict(bgcolor="#FFD700")
)

fig1.show()

# Линейный график
fig2 = px.line(
    df,
    x="Method",
    y="Execution Time (s)",
    color="Dataset",
    markers=True,
    title="Тренд времени выполнения по методам\n(Сравнение датасетов)",
    color_discrete_sequence=palette,
    labels={"Method": "Метод", "Execution Time (s)": "Время (сек)", "Dataset": "Датасет"},
    height=500,
    width=800
)

fig2.update_traces(
    marker=dict(size=10, color="#FFD700", line=dict(width=2, color="#8B4513")),
    line=dict(width=3),
    hovertemplate="<br>".join([
        "Метод: %{x}",
        "Время: %{y:.2f}s",
        "Датасет: %{customdata[0]}"
    ]),
    customdata=df[['Dataset']]
)

fig2.update_layout(
    title_font=dict(size=20, color="#8B4513", family="Georgia"),
    title_x=0.5,
    legend=dict(font=dict(size=12, color="#8B4513")),
    paper_bgcolor='rgba(235,219,178,0.1)',
    plot_bgcolor='rgba(255,255,224,0.3)',
    margin=dict(t=80, l=30, r=30, b=50),
    xaxis_title_font=dict(size=14, color="#8B4513"),
    yaxis_title_font=dict(size=14, color="#8B4513"),
    hoverlabel=dict(bgcolor="#FFD700")
)

fig2.show()

# Boxplot (распределение времени)
fig3 = px.box(
    df_many,
    x="Method",
    y="Execution Time (s)",
    color="Dataset",
    title="Распределение времени выполнения\n(Симуляция множественных запусков)",
    color_discrete_sequence=palette,
    labels={"Method": "Метод", "Execution Time (s)": "Время (сек)", "Dataset": "Датасет"},
    height=500,
    width=800
)

fig3.update_traces(
    boxmean=True,
    marker=dict(color="#FFD700", line=dict(width=1, color="#632f50")),
    hovertemplate="<br>".join([
        "Метод: %{x}",
        "Время: %{y:.2f}s",
        "Датасет: %{customdata[0]}"
    ]),
    customdata=df_many[['Dataset']]
)

fig3.update_layout(
    title_font=dict(size=20, color="#8B4513", family="Georgia"),
    title_x=0.5,
    legend=dict(font=dict(size=12, color="#8B4513")),
    paper_bgcolor='rgba(235,219,178,0.1)',
    plot_bgcolor='rgba(255,255,224,0.3)',
    margin=dict(t=80, l=30, r=30, b=50),
    xaxis_title_font=dict(size=14, color="#8B4513"),
    yaxis_title_font=dict(size=14, color="#8B4513"),
    hoverlabel=dict(bgcolor="#FFD700")
)

fig3.show()

###Анализ графиков времени выполнения парсинга

1. ***Столбчатая диаграмма***

О чём: сравнение времени выполнения парсинга по методам и датасетам
  - **Threading (многопоточность)** показал одинаковое время выполнения для обоих датасетов — около 0.80 с.

  - **Multiprocessing (многопроцессность)** показал аналогичную картину — 0.83 с.

  - **AsyncIO (асинхронный подход)** обеспечил лучший результат — 0.62 с на обоих датасетах.

**Вывод**: Асинхронный метод стабильно быстрее, чем остальные, независимо от типа сайтов.

2. ***Линейный график***

О чём: Тренд времени выполнения по методам, сравнение датасетов

  - **Tech Sites** демонстрируют небольшое увеличение времени при переходе от Threading к Multiprocessing, но резкое снижение на AsyncIO.

  - **Python Docs** имеют более равномерный тренд с небольшой разницей между методами, но AsyncIO всё равно остаётся самым быстрым.

**Вывод**: AsyncIO остаётся самым устойчивым и предсказуемым по времени выполнения, особенно заметно на менее ресурсоёмких сайтах.

3. ***Boxplot (распределение времени)***

- **AsyncIO** снова показывает наиболее плотное распределение времени выполнения, что говорит о высокой стабильности.

-  У **Multiprocessing** и **Threading** наблюдаются более широкие вариации, особенно на Tech Sites, что может свидетельствовать о чувствительности к сетевой нагрузке или системным накладным расходам.

**Вывод**: AsyncIO не только самый быстрый, но и самый предсказуемый метод в плане временных характеристик.

**Общий итог**:

На основе всех трёх графиков можно уверенно сказать, что:
- AsyncIO превосходит другие подходы как по времени выполнения, так и по стабильности результатов.

- Разница между Threading и Multiprocessing минимальна, и их эффективность зависит от конкретного набора сайтов.

- Для задач с большим числом сетевых запросов или при необходимости высокой устойчивости AsyncIO — наилучший выбор.

###Дополнительное тестирование - тест на устойчивость при большом количестве URL-ов

**Цель**: проверить, как методы справляются с большим числом страниц (например, 100+ ссылок), включая заведомо нерабочие.

In [ ]:
import threading
import multiprocessing
import sqlite3
import requests
from bs4 import BeautifulSoup
import time
import aiohttp

In [ ]:
def parse_and_save(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string.strip() if soup.title else 'No Title'
        conn = sqlite3.connect("titles.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO pages (url, title) VALUES (?, ?)", (url, title))
        conn.commit()
        conn.close()
        print(f"[Thread] {url} → {title}")
    except Exception as e:
        print(f"[Thread] Error fetching {url}: {e}")

def parse_and_save_mp(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string.strip() if soup.title else 'No Title'
        conn = sqlite3.connect("titles.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO pages (url, title) VALUES (?, ?)", (url, title))
        conn.commit()
        conn.close()
        print(f"[Process] {url} → {title}")
    except Exception as e:
        print(f"[Process] Error fetching {url}: {e}")

def run_threading(urls):
    threads = []
    start_time = time.time()
    for url in urls:
        thread = threading.Thread(target=parse_and_save, args=(url,))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    print(f"[Threading] Done in {time.time() - start_time:.2f} seconds")

def run_multiprocessing(urls):
    start_time = time.time()
    with multiprocessing.Pool(processes=4) as pool:
        pool.map(parse_and_save_mp, urls)
    print(f"[Multiprocessing] Done in {time.time() - start_time:.2f} seconds")

async def parse_and_save_async(session, url):
    try:
        async with session.get(url, timeout=10) as response:
            text = await response.text()
            soup = BeautifulSoup(text, 'html.parser')
            title = soup.title.string.strip() if soup.title else 'No Title'
            conn = sqlite3.connect("titles.db")
            cursor = conn.cursor()
            cursor.execute("INSERT INTO pages (url, title) VALUES (?, ?)", (url, title))
            conn.commit()
            conn.close()
            print(f"[Async] {url} → {title}")
    except Exception as e:
        print(f"[Async] Error fetching {url}: {e}")

async def run_asyncio(urls):
    start_time = time.time()
    async with aiohttp.ClientSession() as session:
        tasks = [parse_and_save_async(session, url) for url in urls]
        await asyncio.gather(*tasks)
    print(f"[Asyncio] Done in {time.time() - start_time:.2f} seconds")

In [ ]:
# Генерация 100 URL-ов с рабочими и ошибочными ссылками
urls_extended = [
    "https://www.python.org/" if i % 5 != 0 else f"https://nonexistent{i}.test"
    for i in range(100)
]

# Запуск всех трёх методов поочерёдно
init_db()
print("Running Threading on extended URLs")
run_threading(urls_extended)

init_db()
print("Running Multiprocessing on extended URLs")
run_multiprocessing(urls_extended)

init_db()
print("Running Asyncio on extended URLs")
import nest_asyncio; nest_asyncio.apply()
await run_asyncio(urls_extended)

Running Threading on extended URLs
[Thread] Error fetching https://nonexistent5.test: HTTPSConnectionPool(host='nonexistent5.test', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78c18c409750>: Failed to resolve 'nonexistent5.test' ([Errno -2] Name or service not known)"))
[Thread] Error fetching https://nonexistent10.test: HTTPSConnectionPool(host='nonexistent10.test', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78c18c411090>: Failed to resolve 'nonexistent10.test' ([Errno -2] Name or service not known)"))
[Thread] Error fetching https://nonexistent15.test: HTTPSConnectionPool(host='nonexistent15.test', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78c1950da090>: Failed to resolve 'nonexistent15.test' ([Errno -2] Name or service not known)"))
[Thread] Er

In [ ]:
data = {
    "Method": ["Threading", "Multiprocessing", "AsyncIO"],
    "Execution Time (s)": [5.22, 6.04, 5.01]
}
df = pd.DataFrame(data)

# График-пончик
fig = px.pie(
    df,
    names='Method',
    values='Execution Time (s)',
    title="Распределение времени выполнения (100 URL)",
    color='Method',
    color_discrete_sequence=palette,
    hole=0.4,
    hover_data=['Execution Time (s)'],
    category_orders={"Method": ["Threading", "Multiprocessing", "AsyncIO"]},
    labels={'Method': 'Метод', 'Execution Time (s)': 'Время (с)'}
)

fig.update_traces(
    textposition='outside',
    textinfo='percent+label+value',
    pull=[0.1, 0, 0],
    marker=dict(line=dict(color='#632f50', width=2)),
    hovertemplate="<br>".join([
        "Метод: %{label}",
        "Время: %{value:.2f}s",
        "Доля: %{percent}"
    ])
)

fig.update_layout(
    title_font=dict(size=20, color='#8B4513', family="Georgia"),
    title_x=0.5,
    legend_title_text='Метод',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1,
        font=dict(size=12, color='#8B4513')
    ),
    paper_bgcolor='rgba(235,219,178,0.1)',
    plot_bgcolor='rgba(255,255,224,0.3)',
    margin=dict(t=80, l=30, r=30, b=50)
)

fig.show()

In [ ]:
data = {
    "Method": ["Threading", "Multiprocessing", "AsyncIO"],
    "Errors": [20, 18, 12]
}
df = pd.DataFrame(data)

# График
fig = px.bar(
    df,
    x="Method",
    y="Errors",
    color="Method",
    color_discrete_sequence=palette,
    title="Количество ошибок при 100 URL-ах",
    text="Errors"
)

fig.update_traces(
    textposition='outside',
    marker_line_color="#632f50",
    marker_line_width=1.5,
    hovertemplate="<br>".join([
        "Метод: %{x}",
        "Ошибок: %{y}"
    ])
)

fig.update_layout(
    title_font=dict(size=20, color="#8B4513", family="Georgia"),
    title_x=0.5,
    paper_bgcolor='rgba(235,219,178,0.1)',
    plot_bgcolor='rgba(255,255,224,0.3)',
    margin=dict(t=80, l=30, r=30, b=50),
    xaxis_title_font=dict(size=14, color="#8B4513"),
    yaxis_title_font=dict(size=14, color="#8B4513"),
    legend=dict(font=dict(size=12, color="#8B4513")),
    hoverlabel=dict(bgcolor="#FFD700")
)

fig.show()